In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(dplyr)
    library(tibble)
})


In [1]:
source("scTools/R/process.geo.data.r")

In [6]:
seurat_objs <- process.GEO.data(GEOID = "GSE134809")
length(seurat_objs)
seurat_objs

[1] 31

$GSM397200969
An object of class Seurat 
15951 features across 3421 samples within 1 assay 
Active assay: RNA (15951 features, 0 variable features)
 1 layer present: counts

$GSM397201068
An object of class Seurat 
15982 features across 8123 samples within 1 assay 
Active assay: RNA (15982 features, 0 variable features)
 1 layer present: counts

$GSM3972011122
An object of class Seurat 
15225 features across 1438 samples within 1 assay 
Active assay: RNA (15225 features, 0 variable features)
 1 layer present: counts

$GSM3972012123
An object of class Seurat 
15386 features across 2725 samples within 1 assay 
Active assay: RNA (15386 features, 0 variable features)
 1 layer present: counts

$GSM3972013128
An object of class Seurat 
17085 features across 2964 samples within 1 assay 
Active assay: RNA (17085 features, 0 variable features)
 1 layer present: counts

$GSM3972014129
An object of class Seurat 
16012 features across 2378 samples within 1 assay 
Active assay: RNA (16012 features,

In [24]:
# Merge samples
# seurat_objs <- get(load("objects/GSE134809.rda"))
obj <- merge(seurat_objs[[1]], 
c(seurat_objs[[2]], seurat_objs[[3]], seurat_objs[[4]], seurat_objs[[5]], seurat_objs[[6]], seurat_objs[[7]], # nolint: line_length_linter.
  seurat_objs[[8]], seurat_objs[[9]], seurat_objs[[10]], seurat_objs[[11]], seurat_objs[[12]], seurat_objs[[13]],
  seurat_objs[[14]], seurat_objs[[15]], seurat_objs[[16]], seurat_objs[[17]], seurat_objs[[18]], 
  seurat_objs[[19]], seurat_objs[[20]], seurat_objs[[21]], seurat_objs[[22]], seurat_objs[[23]], 
  seurat_objs[[24]], seurat_objs[[25]], seurat_objs[[26]], seurat_objs[[27]], seurat_objs[[28]], 
  seurat_objs[[29]], seurat_objs[[30]], seurat_objs[[31]]))

# Update metadata
obj@meta.data$Stim <- obj@meta.data$status.ch1
obj@meta.data$Tissue <- obj@meta.data$tissue.ch1

obj@meta.data$Donor <- gsub("[^0-9]", "", obj@meta.data$title)
obj@meta.data$Donor <- as.character(obj@meta.data$Donor)

columns_to_keep <-  c('Donor', 'Tissue', 'Stim')
obj@meta.data <- obj@meta.data[, columns_to_keep, drop = FALSE]
head(obj@meta.data)
obj
save(obj, file = file.path("objects", "00.rda"))

Warning message:
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


,Donor,Tissue,Stim
,<chr>,<chr>,<chr>
AAACATACACACCA-1_1,69,ileal,Involved
AAACATTGGTGTCA-1_1,69,ileal,Involved
AAACGCACTTAGGC-1_1,69,ileal,Involved
AAACGCTGCTACCC-1_1,69,ileal,Involved
AAACGCTGCTCATT-1_1,69,ileal,Involved
AAACGCTGTCCAGA-1_1,69,ileal,Involved


An object of class Seurat 
23022 features across 142663 samples within 1 assay 
Active assay: RNA (23022 features, 0 variable features)
 31 layers present: counts.1, counts.2, counts.3, counts.4, counts.5, counts.6, counts.7, counts.8, counts.9, counts.10, counts.11, counts.12, counts.13, counts.14, counts.15, counts.16, counts.17, counts.18, counts.19, counts.20, counts.21, counts.22, counts.23, counts.24, counts.25, counts.26, counts.27, counts.28, counts.29, counts.30, counts.31

In [ ]:
# load(file=file.path("objects", "00.rda"))
options(future.globals.maxSize = 8000 * 1024^2)
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi',
conserve.memory = FALSE, verbose = TRUE)

save(obj, file=file.path("objects", "01.rda"))

In [8]:
# Clean up data
# obj <- JoinLayers(obj)
obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
# obj <- subset(obj, subset = nFeature_RNA > 350 & nFeature_RNA < 5000 & percent.mt < 10)
obj
save(obj, file = file.path("objects", "02.rda"))

An object of class Seurat 
23022 features across 115701 samples within 1 assay 
Active assay: RNA (23022 features, 0 variable features)
 1 layer present: counts

In [ ]:
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()

# Blueprint annnotation
sce <- LayerData(obj)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj@meta.data$blueprint.main <- blueprint.main$pruned.labels
save(obj, file=file.path("objects", "03.rda"))

# Monaco annontation
sce <- LayerData(obj)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj@meta.data$monaco.fine <- monaco.fine$pruned.labels
save(obj, file=file.path('objects', '04.rda'))

In [3]:
# seu =  obj
# seu
# seu[["SCT"]] <- as(object = seu[["SCT"]], Class = "Assay")
# seu

An object of class Seurat 
44058 features across 142663 samples within 2 assays 
Active assay: SCT (21036 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA

Warning message:
“Assay SCT changing from SCTAssay to Assay”


An object of class Seurat 
44058 features across 142663 samples within 2 assays 
Active assay: SCT (21036 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA

In [10]:
# DefaultAssay(obj) <- "RNA"
# obj <- JoinLayers(obj)

In [3]:
load(file=file.path("objects", "04.rda"))

In [4]:
head(obj@meta.data)

,Donor,Tissue,Stim,nCount_SCT,nFeature_SCT,percent.mt,blueprint.main,monaco.fine
,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>
AAACATACACACCA-1_1,NA,ileal,Involved,1294,499,1.3137558,CD8+ T-cells,Th1/Th17 cells
AAACATTGGTGTCA-1_1,NA,ileal,Involved,1540,713,1.0389610,Fibroblasts,Plasmacytoid dendritic cells
AAACGCACTTAGGC-1_1,NA,ileal,Involved,1447,695,0.4837595,Fibroblasts,Plasmacytoid dendritic cells
AAACGCTGCTACCC-1_1,NA,ileal,Involved,1333,625,1.0502626,CD4+ T-cells,T regulatory cells
AAACGCTGCTCATT-1_1,NA,ileal,Involved,1233,358,1.5409570,CD8+ T-cells,Th1/Th17 cells
AAACGCTGTCCAGA-1_1,NA,ileal,Involved,1245,358,2.0080321,B-cells,Non-switched memory B cells


In [6]:
obj@meta.data$Stim[is.na(obj@meta.data$Stim)] <- "PBMC"

[1] "Involved"   "Uninvolved" "PBMC"

In [ ]:
load(file=file.path("objects", "04.rda"))
obj@meta.data$Stim[is.na(obj@meta.data$Stim)] <- "PBMC"
DefaultAssay(obj) <- "RNA"
obj <- JoinLayers(obj)
DefaultAssay(obj) <- "SCT"
obj <- subset(obj, subset = Tissue == "Blood")
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
# save(obj, file=file.path("objects", "03.rda"))

seu =  obj
scConf1 = createConfig(seu)
makeShinyFiles(seu, scConf1, gene.mapping = TRUE, shiny.prefix = "sc1", shiny.dir = "crohns_scRNAseq_GSE134809/", gex.assay = "SCT")

In [8]:
load(file=file.path("objects", "04.rda"))
obj@meta.data$Stim[is.na(obj@meta.data$Stim)] <- "PBMC"
DefaultAssay(obj) <- "RNA"
obj <- JoinLayers(obj)
DefaultAssay(obj) <- "SCT"
obj <- subset(obj, subset = Tissue == "ileal")

In [11]:
lapply(obj@meta.data, unique)

$Donor
[1] NA

$Tissue
[1] "ileal"

$Stim
[1] "Involved"   "Uninvolved"

$nCount_SCT
   [1] 1294 1540 1447 1333 1233 1245 1135 1212 1653 1343 1217 1299 1416 1515
  [15] 1246 1674 1344 1227 1532 1194 1327 1420 1508 1437 1215 1201 1476 1174
  [29] 1442 1226 1330 1521 1502 1222 1140 1290 1219 1259 1525 1173 1298 1570
  [43] 1249 1213 1495 1382 1155 1461 1389 1205 1539 1175 1360 1654 1683 1207
  [57] 1374 1749 1370 1198 1284 1182 1225 1184 1510 1624 1228 1517 1279 1387
  [71] 1367 1199 1229 1221 1307 1668 1208 1280 1409 1243 1195 1206 1448 1469
  [85] 1302 1408 1168 1397 1561 1388 1402 1364 1224 1335 1429 1319 1273 1188
  [99] 1251 1588 1197 1254 1398 1554 1572 1270 1443 1337 1355 1377 1464 1235
 [113] 1310 1269 1356 1466 1170 1318 1436 1527 1751 1415 1384 1362 1559 1593
 [127] 1255 1562 1248 1456 1453 1549 1169 1277 1321 1368 1609 1493 1250 1315
 [141] 1553 1204 1550 1357 1474 1282 1286 1650 1564 1241 1441 1331 1332 1424
 [155] 1924 1239 1346 1481 1278 1341 1156 1275 1154 1451 1230 1487 1475 1689
 [169] 1203 1585 1164 1232 1263 1220 1329 1238 1325 1209 1339 1183 1292 1531
 [183] 1477 1193 1490 1396 1234 1260 1166 1452 1638 1148 1462 1247 1186 1218
 [197] 1444 1372 1681 1262 1667 1211 1739 1317 1214 1516 1266 1509 1412 1190
 [211] 1242 1473 1187 1240 1633 2020 1537 1432 1379 1598 1410 1406 1680 1543
 [225] 1435 1478 1706 1145 1439 1293 1111 1568 1620 1752 1725 1261 1210 1177
 [239] 1285 1366 1334 1257 1328 1696 1748 1505 1545 1185 1267 1287 1161 1607
 [253] 1700 1216 1326 1165 1191 1308 1673 1351 1192 1322 1428 1626 1411 1619
 [267] 1479 1300 1672 1223 1485 1768 1446 1392 1268 1172 1189 1422 1265 1512
 [281] 1702 1258 1399 1468 1709 1353 1202 1679 1455 1576 1655 1264 1454 1574
 [295] 1178 1723 1463 1231 1687 1312 1313 1378 1180 1765 1459 1394 1643 1314
 [309] 1297 1440 1276 1778 1359 1336 1805 1393 1694 1566 1494 1634 1520 1385
 [323] 1295 1557 1676 1622 1648 1591 1395 1491 1556 1430 1403 1371 1662 1179
 [337] 1316 1583 1538 1511 1244 1149 1486 1358 1414 1589 1552 1736 1427 1695
 [351] 1253 1296 1563 1391 1383 1526 1569 1413 1458 1795 1779 1171 1492 1718
 [365] 1675 1176 1649 1352 1236 1340 1417 1472 1489 1792 1274 1500 1375 1271
 [379] 1774 1369 1354 1758 1381 1769 1181 1496 1305 1471 1605 1599 1617 1405
 [393] 1386 1350 1615 1715 1433 1658 1338 1345 1518 1652 1283 1682 1303 1404
 [407] 1272 1659 1281 1363 1380 1686 1482 1507 1361 1200 1438 1671 1717 1431
 [421] 1465 1426 1558 1488 1692 1578 1581 1606 1635 1376 1407 1237 1324 1582
 [435] 1421 1501 1601 1603 1688 1698 1731 1727 1504 1342 1449 1546 1632 1754
 [449] 1613 1252 1450 1503 1150 1763 1418 1645 1323 1555 1288 1304 1470 1401
 [463] 1196 1499 1400 1425 1419 1551 1434 1753 1714 1445 1579 1390 1347 1625
 [477] 1497 1656 1707 1580 1301 1590 1524 1734 1144 1776 1584 1670 1498 1660
 [491] 1506 1721 1480 1630 1740 1642 1289 1612 1839 1467 1167 1722 1152 1640
 [505] 1528 1519 1460 1348 1647 1514 1256 1666 1373 1309 1349 1665 1573 1575
 [519] 1522 1746 1720 1320 1856 1291 1483 1755 1560 1787 1669 1082 1542 1827
 [533] 1457 1744 1586 1143 1594 1724 1794 1631 1311 1789 1741 1843 1636 1533
 [547] 1162 1151 1745 1536 1604 1661 1127 1712 1865 1541 1614 1513 1544 1101
 [561] 1565 1608 1623 1423 1595 1628 1685 1782 1530 1112 1365 1729 1547 1684
 [575] 1535 1548 1121 1163 1738 1716 1126 1772 1637 1600 1306 1879 1678 1750
 [589] 1783 1777 1663 1147 1114 1713 1592 1109 1068 1153 1719 1703 1142 1567
 [603] 1759 1523 2550 2087 2780 2899 2040 1877 2157 2723 1811 2854 2855 2786
 [617] 2752 2002 2859 2805 2038 2825 2886 2065 1998 2567 2405 2149 2523 2785
 [631] 2777 2015 2878 2076 2790 2770 2509 1851 2479 2806 2778 2804 2797 1986
 [645] 2717 2766 2835 2524 2821 2046 2610 2814 1993 2031 2810 2750 3157 2597
 [659] 2250 2735 2338 2563 2909 2595 2860 2781 2226 2582 2652 2639 2951 2624
 [673] 2893 2287 2468 2700 2659 2670 2779 2514 2627 2904 2784 2559 2508 2740
 [687] 2379 2783 2703 1941 2059 2009 2769 2642 2743 2839 2758 2974 2845 2171
 [701] 2414 2228 2727 2118 1985 2482 2388 2706 2533 2628 2751 274

In [ ]:
load(file=file.path("objects", "04.rda"))
obj@meta.data$Stim[is.na(obj@meta.data$Stim)] <- "PBMC"
DefaultAssay(obj) <- "RNA"
obj <- JoinLayers(obj)
DefaultAssay(obj) <- "SCT"
obj <- subset(obj, subset = Tissue == "ileal")
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
# save(obj, file=file.path("objects", "03.rda"))

seu =  obj
scConf2 = createConfig(seu)
makeShinyFiles(seu, scConf2, gene.mapping = TRUE, shiny.prefix = "sc2", shiny.dir = "crohns_scRNAseq_GSE134809/", gex.assay = "SCT")

In [16]:
makeShinyCodesMulti(
  shiny.title = "Crohn's  scRNAseq (GSE134809)", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2"),
  shiny.headers = c( "Blood", "Ileal"), 
  shiny.dir = "crohns_scRNAseq_GSE134809/")

In [25]:
# Launch shiny app
system("R -e \"shiny::runApp('crohns_scRNAseq_GSE134809')\"")